### Pip install

In [ ]:
# that's for Google Colab

In [1]:
#!pip install pennylane --upgrade

     |████████████████████████████████| 756 kB 17.6 MB/s 
     |████████████████████████████████| 305 kB 29.9 MB/s 
     |████████████████████████████████| 108 kB 43.7 MB/s 


## Imports

In [2]:
import numpy 

import pennylane as qml
from pennylane import numpy as np


## Circuit specifications

In [3]:
class QubitsArrangement:
  def __init__(self, nr_trash, nr_latent, nr_swap=1, nr_ent=2):
    self.latent_qubits =[ i for i in range(nr_latent) ] # qubits in register A 
    self.trash_qubits = [ i for i in range(nr_latent,nr_latent+nr_trash) ]  # qubits in register B
    self.aux_qubits = [ i for i in range(nr_latent+nr_trash,nr_latent+2*nr_trash) ]# qubits in a (nr_q(a)=nr_q(B))
    self.swap_qubits = [ i for i in range(nr_latent+2*nr_trash,nr_latent+2*nr_trash+nr_swap) ]#qubits used for swap tests 
    self.ent_qubits = [ i for i in range(nr_latent+2*nr_trash+nr_swap,nr_latent+2*nr_trash+nr_swap+nr_ent) ] #qubits used for entanglement 
    self.qubits = [i for i in range(nr_latent+2*nr_trash+nr_swap+nr_ent)] # list with all the qubits

In [4]:
spec = QubitsArrangement(nr_trash=1, nr_latent=2, nr_swap=1, nr_ent=0)

## Initialization 

In [5]:
# AB initialization #

# Amplitud encoding:
def setAB_amplitude(spec, inputs):
  """
  insputs: vector of amplitudes
  """
  qml.templates.embeddings.AmplitudeEmbedding(inputs, wires = [*spec.latent_qubits,*spec.trash_qubits], normalize = True,pad_with=(0.j))

# Angle encoding:
def setAB_angle(spec, inputs, rotation):
  """
  inputs: vector on angles
  """
  qml.templates.embeddings.AngleEmbedding(inputs, wires =[*spec.latent_qubits,*spec.trash_qubits], rotation=rotation)

# State encoding:
def setAB_state(spec, inputs):
  """
  insputs: state vector 
  """
  qml.MottonenStatePreparation(inputs, wires =[*spec.latent_qubits,*spec.trash_qubits])


# Auxiliary Qubits ialization #

# prepare the state in witch trash-qubits will be reinitialized:
def setAux(spec,inputs):
  """
  insputs: state vector 
  """
  qml.MottonenStatePreparation(inputs, wires = spec.aux_qubits)


# Entanglement preparation #

# Prepare the entagled qubits:
# we can aso decide to use  H CX or something else for the final submmision
# for tests this option is quite eazy to customize 
def setEnt(spec,inputs):
  """
  insputs: state vector 
  """
  qml.MottonenStatePreparation(inputs, wires = spec.ent_qubits)





## Encoder operator 

### encoder

In [6]:
# One of the cicruits from the original paper 

def e1_classic(params,wires):
  """
  nr_params = 2*3*len(wires)+ 3*(len(wires)-1)*len(wires)
  """

  # Add the first rotational gates:
  idx = 0
  for i in wires:
    # qml.Rot(phi, theta, omega, wire)
    qml.Rot(params[idx], params[idx+1], params[idx+2], wires = i)
    idx += 3

  # Add the controlled rotational gates
  for i in wires:
    for j in wires:
      if i!= j:
        qml.CRot(params[idx], params[idx+1], params[idx+2], wires = [i, j])
        idx += 3

  # Add the last rotational gates:
  for i in wires:
    # qml.Rot(phi, theta, omega, wire)
    qml.Rot(params[idx], params[idx+1], params[idx+2], wires = i)
    idx += 3



In [7]:
# we need totest more circuit configurations

### decoder

In [8]:
# this decodor is basiclay the adjoint of the encoder
def decoder_adjoint(encoder_circuit,params, wires):
  return qml.adjoint(encoder_circuit)(params, wires)

# parametrizad dencoders 
def d1_classic(params, wires):
  """
  nr_params = 2*3*len(wires)+ 3*(len(wires)-1)*len(wires)
  """

   # Add the first rotational gates:
  idx = 0
  for i in wires:
    # qml.Rot(phi, theta, omega, wire)
    qml.Rot(params[idx], params[idx+1], params[idx+2], wires = i)
    idx += 3

  # Add the controlled rotational gates
  for i in wires:
    for j in wires:
      if i!= j:
        qml.CRot(params[idx], params[idx+1], params[idx+2], wires = [i, j])
        idx += 3

  # Add the last rotational gates:
  for i in wires:
    # qml.Rot(phi, theta, omega, wire)
    qml.Rot(params[idx], params[idx+1], params[idx+2], wires = i)
    idx += 3



In [9]:
# we need totest more circuit configurations

## Swap test


In [15]:
def swap_t(spec):
  for i in spec.swap_qubits:
    qml.Hadamard(wires = i)
  for i in range(len(spec.trash_qubits)):
    qml.CSWAP(wires = [*spec.swap_qubits, spec.aux_qubits[i], spec.trash_qubits[i] ])
  for i in spec.swap_qubits:
    qml.Hadamard(wires = i)

## Additional evaluation methods

-> 

In [11]:
##

## Training node


In [12]:
spec = QubitsArrangement(nr_trash=2, nr_latent=2, nr_swap=1, nr_ent=2)
print("Qubtis:", spec.qubits)

#set up the device 
dev = qml.device("default.qubit", wires=len(spec.qubits))

Qubtis: [0, 1, 2, 3, 4, 5, 6, 7, 8]


In [13]:

@qml.qnode(dev)
def training_circuit_example(init_params, encoder_params,reinit_state):

  #initilaization 
  setAB_amplitude(spec, init_params)

  setAux(spec,reinit_state)

  setEnt(spec,inputs=[1/np.sqrt(2),0,0,1/np.sqrt(2)])  



  #encoder 
  for params in encoder_params:
    e1_classic(params,  [*spec.latent_qubits,*spec.trash_qubits])

  #swap test 
  swap_t(spec)

  return  [qml.probs(i) for i in spec.swap_qubits]


In [16]:
nr_encod_qubits = len(spec.trash_qubits)+len(spec.latent_qubits)
nr_par_encoder = 2*3*nr_encod_qubits+ 3*(nr_encod_qubits-1)*nr_encod_qubits
encoder_params= np.random.uniform(size=(1, nr_par_encoder), requires_grad=True)

init_params = [0 for i in range(2**nr_encod_qubits)]
init_params[2] = 1


reinit_state =  [0 for i in range(2**len(spec.aux_qubits))]
reinit_state[0] = 1



print(qml.draw(training_circuit_example)(init_params, encoder_params,reinit_state))

 0: ──╭AmplitudeEmbedding(M0)────────Rot(0.793, 0.759, 0.483)───╭C─────────────────────────╭C──────────────────────────╭C──────────────────────────╭Rot(0.521, 0.152, 0.282)───────────────────────────────────────────────────────╭Rot(0.838, 0.543, 0.669)─────────────────────────────────────────────────────╭Rot(0.324, 0.81, 0.0763)───Rot(0.0322, 0.0998, 0.879)──────────────────────────────────────────────────────────────────────────┤       
 1: ──├AmplitudeEmbedding(M0)────────Rot(0.881, 0.13, 0.321)────╰Rot(0.267, 0.032, 0.211)──│───────────────────────────│───────────────────────────╰C─────────────────────────╭C─────────────────────────╭C────────────────────────│──────────────────────────╭Rot(0.686, 0.907, 0.45)───────────────────────────│──────────────────────────╭Rot(0.0471, 0.614, 0.988)────Rot(0.913, 0.743, 0.877)───────────────────────────────────────────────┤       
 2: ──├AmplitudeEmbedding(M0)────────Rot(0.464, 0.284, 0.503)──────────────────────────────╰Rot(0.901, 0.392, 0.0331

## Training example 

# Check the following notebook: 
-> https://colab.research.google.com/drive/1QCmp8uCqLXslk0FaGRlgRJW5qrPyhU14?usp=sharing